In [ ]:
from preprocess import *

extract_gps_from_video("./download/20220413_101753.mp4")

In [ ]:
[
    {
        "SourceFile": "/Users/samyuktha/pixtale/download/20220413_101753.mp4",
        "ExifTool:ExifToolVersion": 12.76,
        "File:Directory": "/Users/samyuktha/pixtale/download",
        "File:FileAccessDate": "2024:04:23 23:03:10+05:30",
        "File:FileInodeChangeDate": "2024:04:23 23:03:10+05:30",
        "File:FileModifyDate": "2024:04:23 23:03:10+05:30",
        "File:FileName": "20220413_101753.mp4",
        "File:FilePermissions": "-rw-r--r--",
        "File:FileSize": "6.1 MB",
        "File:FileType": "MP4",
        "File:FileTypeExtension": "mp4",
        "File:MIMEType": "video/mp4",
        "QuickTime:AudioBitsPerSample": 16,
        "QuickTime:AudioChannels": 2,
        "QuickTime:AudioFormat": "mp4a",
        "QuickTime:AudioSampleRate": 44100,
        "QuickTime:Balance": 0,
        "QuickTime:BitDepth": 24,
        "QuickTime:CompatibleBrands": ["isom", "mp42"],
        "QuickTime:CompressorID": "avc1",
        "QuickTime:CreateDate": "0000:00:00 00:00:00",
        "QuickTime:CurrentTime": "0 s",
        "QuickTime:Duration": "5.06 s",
        "QuickTime:Encoder": "Google",
        "QuickTime:GraphicsMode": "srcCopy",
        "QuickTime:HandlerDescription": "ISO Media file produced by Google Inc.",
        "QuickTime:HandlerType": "Metadata",
        "QuickTime:HandlerVendorID": "Apple",
        "QuickTime:ImageHeight": 1920,
        "QuickTime:ImageWidth": 1080,
        "QuickTime:MajorBrand": "MP4 v2 [ISO 14496-14]",
        "QuickTime:MatrixStructure": "1 0 0 0 1 0 0 0 1",
        "QuickTime:MediaCreateDate": "0000:00:00 00:00:00",
        "QuickTime:MediaDataOffset": 3000,
        "QuickTime:MediaDataSize": 6138815,
        "QuickTime:MediaDuration": "5.06 s",
        "QuickTime:MediaHeaderVersion": 0,
        "QuickTime:MediaLanguageCode": "eng",
        "QuickTime:MediaModifyDate": "0000:00:00 00:00:00",
        "QuickTime:MediaTimeScale": 44100,
        "QuickTime:MinorVersion": "0.0.0",
        "QuickTime:ModifyDate": "0000:00:00 00:00:00",
        "QuickTime:MovieHeaderVersion": 0,
        "QuickTime:NextTrackID": 3,
        "QuickTime:OpColor": "0 0 0",
        "QuickTime:PosterTime": "0 s",
        "QuickTime:PreferredRate": 1,
        "QuickTime:PreferredVolume": "100.00%",
        "QuickTime:PreviewDuration": "0 s",
        "QuickTime:PreviewTime": "0 s",
        "QuickTime:SelectionDuration": "0 s",
        "QuickTime:SelectionTime": "0 s",
        "QuickTime:SourceImageHeight": 1920,
        "QuickTime:SourceImageWidth": 1080,
        "QuickTime:TimeScale": 1000,
        "QuickTime:TrackCreateDate": "0000:00:00 00:00:00",
        "QuickTime:TrackDuration": "5.00 s",
        "QuickTime:TrackHeaderVersion": 0,
        "QuickTime:TrackID": 1,
        "QuickTime:TrackLayer": 0,
        "QuickTime:TrackModifyDate": "0000:00:00 00:00:00",
        "QuickTime:TrackVolume": "0.00%",
        "QuickTime:VideoFrameRate": 29.97,
        "QuickTime:XResolution": 72,
        "QuickTime:YResolution": 72,
        "Composite:AvgBitrate": "9.7 Mbps",
        "Composite:ImageSize": "1080x1920",
        "Composite:Megapixels": 2.1,
        "Composite:Rotation": 0,
    }
]

In [ ]:


mediaitems = extract_metadata("./download")

In [ ]:


import json

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


mediaitems = generate_descriptions(dir_path, mediaitems)




In [ ]:
mediaitems.sort(key=lambda item: item["datetime"])

mediaitems

In [ ]:
import json
from geminiai import *

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


narration_script = get_script(mediaitems)

narration_script

In [ ]:
import shutil
from moviepy.editor import (
    VideoFileClip,
    ImageClip,
    concatenate_videoclips,
    TextClip,
    CompositeVideoClip,
    concatenate_audioclips,
    AudioFileClip,
)
from moviepy.video.fx.resize import resize
import os
from moviepy.video import fx as vfx
from moviepy.video.fx import speedx


clips = []

# Define a standard resolution
standard_resolution = (1080, 1920)  # Width x Height


os.makedirs("./audio")

audio_clips = []
# Process each scene
for i in range(len(narration_script["scenes"])):
    scene = narration_script["scenes"][i]

    text = scene["text"]

    filepath = os.path.join(dir_path, scene["media_source"]).lower()

    print(filepath)

    if os.path.exists(filepath):

        if not os.path.exists("./audio/" + str(i) + ".mp3"):

            synthesize_text(text, "./audio/" + str(i))

        audio_clip = AudioFileClip("./audio/" + str(i) + ".mp3")

        duration = audio_clip.duration

        if filepath.endswith((".mp4", ".mov")):
            clip = VideoFileClip(filepath)

            if duration < clip.duration:

                clip = clip.subclip(0, duration)
                
            clip=clip.resize(newsize=(standard_resolution[0], standard_resolution[1]))

        else:
            clip = ImageClip(filepath).set_duration(duration)

        # txt_clip = (
        #     TextClip(
        #         scene["narration"]["text"],
        #         color="white",
        #         font="Arial",
        #         fontsize=50,
        #         method="caption",
        #         stroke_width=2,
        #     )
        #     .set_position("bottom")
        #     .set_duration(float(duration))
        # )
        # clip = CompositeVideoClip([clip, txt_clip])
        clips.append(clip)

        audio_clips.append(audio_clip)


# shutil.rmtree("./audio")
# Concatenate all clips
final_clip = concatenate_videoclips(clips, method="compose")

final_audio_clip = concatenate_audioclips(audio_clips)

final_audio_clip.write_audiofile("final_trip_audio.mp3")

if final_clip.duration != final_audio_clip.duration:

    if final_clip.duration > final_audio_clip.duration:
        # Speed up the video clip to match the audio clip's duration
        speed_factor = final_clip.duration / final_audio_clip.duration
        final_clip = final_clip.fx(vfx.speedx, speed_factor)
    else:
        # Speed up the audio clip to match the video clip's duration

        speed_factor = final_audio_clip.duration / final_clip.duration
        print(speed_factor)
        cmd = (
            'ffmpeg -y -i final_trip_audio.mp3 -filter:a "atempo={}" output.mp3'.format(
                speed_factor
            )
        )

        os.system(cmd)


# Write the result to a file
final_clip.write_videofile("final_trip_video.mp4")

In [ ]:
cmd = "ffmpeg -y -i final_trip_video.mp4 -i output.mp3 -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac combined_video.mp4"
os.system(cmd)

In [ ]:
def list_voices():
    """Lists the available voices."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    voices = client.list_voices()

    english_voices = []

    for voice in voices.voices:

        for language_code in voice.language_codes:
            if "en" in language_code:

                english_voices.append(
                    {
                        "language_codes": voice.language_codes[0],
                        "ssml_gender": texttospeech.SsmlVoiceGender(
                            voice.ssml_gender
                        ).name,
                        "name": voice.name,
                        "voice": "{}-{}".format(
                            voice.name,
                            texttospeech.SsmlVoiceGender(voice.ssml_gender).name,
                        ),
                    }
                )

    return english_voices

print(list_voices())

In [ ]:
from utils import *

public = "./static/public"
with open("data/narration_script.json", "r") as json_file:
    narration_script = json.load(json_file)

create_video(narration_script,public)

In [ ]:
import json

from utils import *


with open("data/mediaitems.json", "r") as file:
    mediaitems = json.load(file)

create_video(mediaitems, "./static/public")

In [ ]:
from ai_utils import *



In [ ]:
import os
from utils import *
import shutil
from moviepy.editor import (
    VideoFileClip,
    ImageClip,
    concatenate_videoclips,
    TextClip,
    CompositeVideoClip,
    concatenate_audioclips,
    AudioFileClip,
)
from moviepy.video.fx.resize import resize
import os
from moviepy.video import fx as vfx
from moviepy.video.fx import speedx
from ai_utils import *

def get_audio_duration(filepath):
    """Get the duration of an audio file using ffprobe with os.system."""
    # Temporary file to store the output of ffprobe
    temp_file = "temp_duration.txt"
    cmd = f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 {filepath} > {temp_file}"
    os.system(cmd)

    # Read the duration from the temporary file
    with open(temp_file, "r") as file:
        duration = file.read().strip()

    # Remove the temporary file
    os.remove(temp_file)

    try:
        return float(duration)
    except ValueError:
        return None

import os


def get_video_duration(filepath):
    """Get the duration of a video file using ffprobe with os.system."""
    # Temporary file to store the output of ffprobe
    temp_file = "temp_duration.txt"
    cmd = f"ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 {filepath} > {temp_file}"
    os.system(cmd)

    # Read the duration from the temporary file
    with open(temp_file, "r") as file:
        duration = file.read().strip()

    # Remove the temporary file
    os.remove(temp_file)

    try:
        return float(duration)
    except ValueError:
        return None


# Usage example:
# duration = get_video_duration_using_os_system("/path/to/your/video.mp4")


In [ ]:

def generate_video(mediaitems, dir_path):
    image_extensions = (".jpg", ".jpeg", ".png", ".gif", ".heic")
    video_extensions = (".mov", ".mp4", ".avi", ".flv", ".wmv")
    # Ensure the result directory exists
    result_dir = os.path.join(dir_path, "result")

    print("result directory: ", result_dir)

    os.makedirs(result_dir, exist_ok=True)

    audio_dir=os.path.join(dir_path, "result/audio")

    os.makedirs(audio_dir, exist_ok=True)

    # File to store list of files to concatenate

    filelist_path = os.path.join(result_dir, "filelist.txt")
    audio_clips=[]
    with open(filelist_path, "w+") as filelist:
        for i, item in enumerate(mediaitems):
            filepath = os.path.join(dir_path, item["filename"]).lower()

            print("file: ", filepath)

            audio_path = os.path.join(audio_dir, str(i) + ".mp3")

            if not os.path.exists(audio_path):

                synthesize_text(item["narration_text"], os.path.join(audio_dir, str(i)))

            audio_duration=get_audio_duration(audio_path)
            audio_clip = AudioFileClip(audio_path)
            audio_clips.append(audio_clip)

            if filepath.lower().endswith(image_extensions):
                # Convert image to video clip
                video_clip_path = os.path.join(result_dir, f"clip_{i}.mp4")
                cmd = f"ffmpeg -y -loop 1 -i {filepath} -c:v libx264 -t {audio_duration} -pix_fmt yuv420p -vf \"scale=1080:1920:force_original_aspect_ratio=decrease,pad=1080:1920:(ow-iw)/2:(oh-ih)/2\" {video_clip_path}"
                os.system(cmd)
            elif filepath.lower().endswith(video_extensions):
                # Ensure video clip is of the correct format and resolution
                video_clip_path = os.path.join(result_dir, f"clip_{i}.mp4")
                video_duration=get_video_duration(filepath)
                cmd = f"ffmpeg -y -i {filepath} -c:v libx264 -t {video_duration} -pix_fmt yuv420p -vf scale=1080:1920 {video_clip_path}"
                os.system(cmd)

            output_clip_path = os.path.join(result_dir, f"output_clip_{i}.mp4")
            cmd = f"ffmpeg -y -i {video_clip_path} -i {audio_path} -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac {output_clip_path}"
            os.system(cmd)
            filelist.write(f"file 'output_clip_{i}.mp4'\n")
    combined_video_path = os.path.join(result_dir, "combined_video.mp4")
    cmd = f"ffmpeg -y -f concat -safe 0 -i {filelist_path} -c:v libx264 -pix_fmt yuv420p -crf 23 -preset medium {combined_video_path}"
    os.system(cmd)       


import json


with open("data/mediaitems.json", "r") as file:
    mediaitems = json.load(file)
# Call the function with the media items and directory path
generate_video(mediaitems, "./static/public")

In [ ]:
from google_auth import *


#get_all_albums()

In [ ]:
album_id = "AG59PP9EfmZZr_dVebhxoyKPBO92WbMvkf8Z7zAzbtWb5QK-YfYad7dzTxMhA8Eta0ZUBQdKozPQ"


download_album_items(album_id,"./download")


In [9]:
import os
from utils import *
import shutil
from moviepy.editor import (
    VideoFileClip,
    ImageClip,
    concatenate_videoclips,
    TextClip,
    CompositeVideoClip,
    concatenate_audioclips,
    AudioFileClip,
)
from moviepy.video.fx.resize import resize
import os
from moviepy.video import fx as vfx
from moviepy.video.fx import speedx
from ai_utils import *


def generate_video(mediaitems, dir_path, global_voice):

    print("global voice", global_voice)
    image_extensions = (".jpg", ".jpeg", ".png", ".gif", ".heic")
    video_extensions = (".mov", ".mp4", ".avi", ".flv", ".wmv")
    # Ensure the result directory exists
    result_dir = os.path.join(dir_path, "result")

    print("result directory: ", result_dir)

    os.makedirs(result_dir, exist_ok=True)

    audio_dir = os.path.join(dir_path, "result/audio")

    os.makedirs(audio_dir, exist_ok=True)

    # File to store list of files to concatenate

    filelist_path = os.path.join(result_dir, "filelist.txt")
    audio_clips = []
    with open(filelist_path, "w+") as filelist:
        for i, item in enumerate(mediaitems):
            filepath = os.path.join(dir_path, item["filename"]).lower()

            print("file: ", filepath)

            audio_path = os.path.join(audio_dir, str(i) + ".mp3")

            if not os.path.exists(audio_path):

                synthesize_text(
                    item["narration_text"],
                    os.path.join(audio_dir, str(i)),
                    language_code=global_voice["language_codes"],
                    name=global_voice["name"],
                    gender=global_voice["ssml_gender"],
                )

            audio_duration = get_audio_duration(audio_path)
            audio_clip = AudioFileClip(audio_path)
            audio_clips.append(audio_clip)

            if filepath.lower().endswith(image_extensions):
                # Convert image to video clip
                video_clip_path = os.path.join(result_dir, f"clip_{i}.mp4")
                cmd = f'ffmpeg -y -loop 1 -i {filepath} -c:v libx264 -t {audio_duration} -pix_fmt yuv420p -vf "scale=1080:1920:force_original_aspect_ratio=decrease,pad=1080:1920:(ow-iw)/2:(oh-ih)/2" {video_clip_path}'
                os.system(cmd)
            elif filepath.lower().endswith(video_extensions):
                # Ensure video clip is of the correct format and resolution
                video_clip_path = os.path.join(result_dir, f"clip_{i}.mp4")
                video_duration = get_video_duration(filepath)
                cmd = f"ffmpeg -y -i {filepath} -c:v libx264 -t {video_duration} -pix_fmt yuv420p -vf scale=1080:1920 {video_clip_path}"
                os.system(cmd)
            output_clip_path = os.path.join(result_dir, f"output_clip_{i}.mp4")

            filelist.write(f"file 'clip_{i}.mp4'\n")
    combined_video_path = os.path.join(result_dir, "combined_video.mp4")
    cmd = f"ffmpeg -y -f concat -safe 0 -i {filelist_path} -c:v libx264 -pix_fmt yuv420p -crf 23 -preset medium {combined_video_path}"
    os.system(cmd)   
    combined_audio_duration = sum([clip.duration for clip in audio_clips])

    combined_video_duration = get_video_duration(combined_video_path)

    print(combined_video_duration)

    combined_audio_path = os.path.join(result_dir, "combined_audio.mp4")
    concatenate_audioclips(audio_clips).write_audiofile(combined_audio_path, codec="aac")

    if combined_audio_duration > combined_video_duration:
        speedup_factor = combined_audio_duration / combined_video_duration
        spedup_audio_path = os.path.join(result_dir, "spedup_audio.mp4")
        cmd = f"ffmpeg -y -i {combined_audio_path} -filter:a atempo={speedup_factor} {spedup_audio_path}"
        os.system(cmd)
        combined_audio_path = (
            spedup_audio_path  # Update the audio path to use the sped-up version
        )
    elif combined_video_duration > combined_audio_duration:
        speedup_factor = combined_video_duration / combined_audio_duration
        spedup_video_path = os.path.join(result_dir, "spedup_video.mp4")
        cmd = f"ffmpeg -y -i {combined_video_path} -filter:v setpts={1/speedup_factor}*PTS {spedup_video_path}"
        os.system(cmd)
        combined_video_path = spedup_video_path

    # Combine the final audio and video into a single file
    final_output_path = os.path.join(result_dir, "final_output.mp4")
    cmd = f"ffmpeg -y -i {combined_video_path} -i {combined_audio_path} -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac  {final_output_path}"
    os.system(cmd)


with open("data/mediaitems.json", "r") as file:
    mediaitems = json.load(file)
# Call the function with the media items and directory path
generate_video(mediaitems, "./static/public",None)

global voice None
result directory:  ./static/public/result
file:  ./static/public/img_6423.mov


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e

file:  ./static/public/img_6431.mov


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e

file:  ./static/public/img_6425.mov


...

ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e

64.971882
MoviePy - Writing audio in ./static/public/result/combined_audio.mp4


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers      
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

MoviePy - Done.


[out#0/mp4 @ 0x11de40000] video:0KiB audio:1011KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.161090%
size=    1023KiB time=00:01:04.98 bitrate= 129.0kbits/s speed=48.2x    
[aac @ 0x11de3d0b0] Qavg: 1031.990
ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --